In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Step 1 : Dealing with missing values

In [2]:
# TODO Replace with your own path to the csv file before running
path_to_original_csv = "../Datasets/DOB_Job_Application_Filings.csv"

jobs_df = pd.read_csv(path_to_original_csv)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,6,13,14,21,32,38,59,60,68,78,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
jobs_df.head()
display(jobs_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1756231 entries, 0 to 1756230
Data columns (total 96 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Job #                          int64  
 1   Doc #                          int64  
 2   Borough                        object 
 3   House #                        object 
 4   Street Name                    object 
 5   Block                          object 
 6   Lot                            object 
 7   Bin #                          int64  
 8   Job Type                       object 
 9   Job Status                     object 
 10  Job Status Descrp              object 
 11  Latest Action Date             object 
 12  Building Type                  object 
 13  Community - Board              object 
 14  Cluster                        object 
 15  Landmarked                     object 
 16  Adult Estab                    object 
 17  Loft Board                     object 
 18  Ci

None

In [4]:
rem = 0
print("Number of rows before cleaning    : {}".format(jobs_df.shape[0]))
print("Number of columns before cleaning : {}".format(jobs_df.shape[1]))
for col in jobs_df.columns:
    if jobs_df[col].isna().sum() >0:
        rem += 1
        print("Column name, type : {}, {} : values(s) missing".format(col, jobs_df[col].dtypes, jobs_df[col].isna().sum()))
print("Remaining columns to clean : {}".format(rem))

Number of rows before cleaning    : 1756231
Number of columns before cleaning : 96
Column name, type : House #, object : values(s) missing
Column name, type : Street Name, object : values(s) missing
Column name, type : Block, object : values(s) missing
Column name, type : Lot, object : values(s) missing
Column name, type : Latest Action Date, object : values(s) missing
Column name, type : Community - Board, object : values(s) missing
Column name, type : Cluster, object : values(s) missing
Column name, type : Landmarked, object : values(s) missing
Column name, type : Adult Estab, object : values(s) missing
Column name, type : Loft Board, object : values(s) missing
Column name, type : City Owned, object : values(s) missing
Column name, type : Little e, object : values(s) missing
Column name, type : PC Filed, object : values(s) missing
Column name, type : eFiling Filed, object : values(s) missing
Column name, type : Plumbing, object : values(s) missing
Column name, type : Mechanical, obje

In [5]:
# Cleaning Job types columns :
job_types = ["Plumbing", "Mechanical", "Boiler", "Fuel Burning", "Fuel Storage", "Standpipe", 
             "Sprinkler", "Fire Alarm", "Equipment", "Fire Suppression", "Curb Cut", "Other"]

for job_type in job_types:
    jobs_df[job_type] = jobs_df[job_type].map(lambda val: 1 if val == "X" else 0)

In [6]:
# Cleaning Binary variables
jobs_df["City Owned"] = jobs_df["City Owned"].map(lambda val: 1 if val == "Y" else 0)
jobs_df["Landmarked"] = jobs_df["Landmarked"].map(lambda val: 1 if val == "Y" else 0)
jobs_df["Adult Estab"] = jobs_df["Adult Estab"].map(lambda val: 0 if val == "N" else 1)
jobs_df["Little e"] = jobs_df["Little e"].map(lambda val: 1 if val == "H" else 0)

jobs_df["Zoning Dist1"] = jobs_df["Zoning Dist1"].map(lambda val: 0 if pd.isnull(val)  else 1) # Residence
jobs_df["Zoning Dist2"] = jobs_df["Zoning Dist2"].map(lambda val: 0 if pd.isnull(val)  else 1) # Commercial
jobs_df["Zoning Dist3"] = jobs_df["Zoning Dist3"].map(lambda val: 0 if pd.isnull(val)  else 1) # Manufacturing

jobs_df["Special District 1"] = jobs_df["Special District 1"].map(lambda val: 1 if not pd.isnull(val) else 0)
jobs_df["Special District 2"] = jobs_df["Special District 2"].map(lambda val: 1 if not pd.isnull(val) else 0)
jobs_df["Special District"] = jobs_df["Special District 1"] + jobs_df["Special District 2"]
jobs_df["Special District"] = jobs_df["Special District"].map(lambda val: 0 if val == 0 else 1)

# Cleaning missing values in other columns
jobs_df["Vertical Enlrgmt"] = jobs_df["Vertical Enlrgmt"].map(lambda val: 0 if pd.isnull(val) or val == "N" else 1)
jobs_df["Horizontal Enlrgmt"] = jobs_df["Horizontal Enlrgmt"].map(lambda val: 0 if pd.isnull(val) or val == "N" else 1)
jobs_df["Other Description"] = jobs_df["Other Description"].map(lambda desc: desc if not pd.isnull(desc) else "")

In [7]:
# Dropping uniteresting columns
useless_columns = ["PC Filed", "eFiling Filed", "Cluster", "Loft Board", "Non-Profit", "Other Description",
                   "Applicant's First Name", "Applicant's Last Name", "Applicant Professional Title", "Applicant License #",
                   "Professional Cert", "Professional Cert", "Owner's First Name", "Owner's Last Name", "Owner's Business Name",
                   "Owner's House Number", "Owner'sHouse Street Name","Existing Dwelling Units", "Proposed Dwelling Units",
                   "City ", "State", "Zip", "Owner'sPhone #", "BUILDING_CLASS", "Job Description", "SIGNOFF_DATE", "Assigned",
                   "Approved", "Fully Permitted", "Existing Occupancy", "Proposed Occupancy", "SPECIAL_ACTION_DATE", 
                   "SPECIAL_ACTION_STATUS", "Site Fill", "Special District 1", "Special District 2", "DOBRunDate"]

print(len(useless_columns))

jobs_df = jobs_df.drop(labels = useless_columns, axis = 1)

37


In [8]:
# Deletes rows with na values on these columns:
columns_drop_na = ["House #", "Street Name", "Block", "Lot", "GIS_LATITUDE", "GIS_LONGITUDE",
                   "GIS_COUNCIL_DISTRICT", "GIS_CENSUS_TRACT", "GIS_NTA_NAME", "GIS_BIN", "Latest Action Date",
                   "Community - Board", "Paid", "Fully Paid", "Owner Type"]

jobs_df = jobs_df.dropna(subset = columns_drop_na)

In [9]:
# Removing the rows for which we don't know the initial cost
jobs_df = jobs_df[jobs_df['Initial Cost']!="$0.00"]

In [10]:
# Converting prices to floats
jobs_df["Initial Cost"] = jobs_df["Initial Cost"].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(np.float64)
jobs_df["Total Est. Fee"] = jobs_df["Total Est. Fee"].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(np.float64)

In [11]:
# Dates to datetime format
dates_columns = ["Paid", "Fully Paid","Pre- Filing Date", "Latest Action Date"]

for c in dates_columns : 
    jobs_df[c] = pd.to_datetime(jobs_df[c])

In [12]:
# # Horizontal Enlrgmt to dummy variables
# l = list(data["Horizontal Enlrgmt"])
# for i in range(len(l)):
#     if l[i]=='Y':
#         l[i]=1
#     elif l[i]=='N':
#         l[i]=0
#     elif l[i]=='0':
#         l[i]=0
# data["Horizontal Enlrgmt"] = pd.Series(l)

In [13]:
rem = 0
print("Number of rows after cleaning    : {}".format(jobs_df.shape[0]))
print("Number of columns after cleaning : {}".format(jobs_df.shape[1]))
for col in jobs_df.columns:
    if jobs_df[col].isna().sum() >0:
        rem += 1
        print("Column name, type : {}, {} : values(s) missing".format(col, jobs_df[col].dtypes, jobs_df[col].isna().sum()))


Number of rows after cleaning    : 1222083
Number of columns after cleaning : 61


# Step 2 : Adjusting prices for inflation

In [14]:
# Loading the buildings metadata
cpi_path = "../Datasets/CPI.csv"
cpi_df = pd.read_csv(cpi_path)

In [16]:
# Adjusting prices based on inflation

# Gets the CPI of January 2020
CPI_Jan_2020 = cpi_df.loc[252, "Value"]

def clean_cost(cost, year, month):
    
    try:
        month = "M{}".format(month) if len(str(month)) == 2 else "M0{}".format(month)
        if len(month) == 1:
            month = "0" + month
        CPI_then = cpi_df.loc[(cpi_df["Year"] == year) & (cpi_df["Period"] == month), "Value"].iloc[0]
        return cost * CPI_Jan_2020 / CPI_then
    except:
        print("Error y-m : {}-{}".format(year, month))
        return 0
    

jobs_df = jobs_df.reset_index()

l = list(jobs_df["Initial Cost"])

for i in range(len(l)):
    cost = jobs_df.loc[i, "Initial Cost"]
    year = jobs_df.loc[i, "Pre- Filing Date"].year
    month = jobs_df.loc[i, "Pre- Filing Date"].month
    l[i] = clean_cost(cost, year, month)
jobs_df["Initial Cost"] = pd.Series(l)


Error y-m : 2020-M11
Error y-m : 1996-M11
Error y-m : 1996-M11
Error y-m : 1994-M08
Error y-m : 1998-M08
Error y-m : 1995-M02
Error y-m : 1995-M08
Error y-m : 1995-M11
Error y-m : 1996-M08
Error y-m : 1996-M11
Error y-m : 1996-M10
Error y-m : 1997-M04
Error y-m : 1997-M05
Error y-m : 1997-M10
Error y-m : 1998-M04
Error y-m : 1998-M04
Error y-m : 1998-M06
Error y-m : 1992-M08
Error y-m : 1993-M01
Error y-m : 1994-M07
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2

Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2

Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2

Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2

Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2

Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2020-M11
Error y-m : 2

In [17]:
jobs_df = jobs_df[jobs_df['Initial Cost']!= 0]

# Step 3 : Merging with the building info dataset

In [18]:
# Loading the buildings metadata
building_path = "../Datasets/pluto_20v6.csv"
buildings_df = pd.read_csv(building_path)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (19,20,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
buildings_df = buildings_df.filter(items = ['address',"bldgclass","landuse","bldgarea","numfloors","unitsres","unitstotal","bldgfront","bldgdepth","lottype","assesstot","yearbuilt","yearalter1","yearalter2","firm07_flag","lot",'block','ct2010'])
buildings_df['firm07_flag'] = buildings_df['firm07_flag'].fillna(0)


In [20]:
# Merging the cleaned jobs dataset with the buildings metadata
final_df = pd.merge(buildings_df, jobs_df, how = "right", left_on = ['ct2010','lot','block'], right_on = ['GIS_CENSUS_TRACT','Lot','Block'])

# Dropping rows with missing values
final_df = final_df.dropna()

final_df = final_df.loc[:, final_df.columns != 'Unnamed: 0']

In [21]:
output_path = "../Datasets/Jobs_and_buildings_merged_v2.0.csv"
final_df.to_csv(output_path)